# Data and Query Task (Basic)

Insert Library

In [1]:
# Import required libraries
from faker import Faker
from faker.providers import address
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import pyodbc

## 1. Create Script to Create Table

Create Tabel Database Azure

In [2]:
## connect to db azure
server = 'azure-ragun.database.windows.net'
database = 'azure_db'
username = 'azure@azure-ragun'
password = 'Ragun123@'
connection_string = f"Driver={{ODBC Driver 17 for SQL Server}};Server={server};Database={database};Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
conn = pyodbc.connect(connection_string)

In [3]:
def create_table(cursor, table_name, columns):
    """
    Creates a new table in the database.

    Args:
        cursor (pyodbc.Cursor): The cursor object to execute the SQL query.
        table_name (str): The name of the table to be created.
        columns (str): A string representing the column definitions for the table.

    Returns:
        None: The function doesn't return any value.

    Raises:
        pyodbc.Error: If an error occurs while executing the SQL query.

    Example:
        create_table(cursor, "Employee", "Id INT, EmployeeId VARCHAR(10), FullName VARCHAR(100), BirthDate DATE, Address VARCHAR(100)")

    """
    # Construct the CREATE TABLE query
    create_query = f"CREATE TABLE {table_name} ({columns})"

    # Execute the CREATE TABLE query
    cursor.execute(create_query)

    # Print a success message
    print(f"Table {table_name} created.")

    # Commit the changes to the database
    conn.commit()

In [4]:
def drop_table(cursor, table_name):
    """
    Drops the specified table from the database if it exists.

    Args:
        cursor (pyodbc.Cursor): The cursor object to execute the SQL query.
        table_name (str): The name of the table to be dropped.

    Returns:
        None: The function doesn't return any value.

    Raises:
        pyodbc.Error: If an error occurs while executing the SQL query.

    Example:
        drop_table(cursor, "PositionHistory")

    """
    # Check if the table exists
    check_query = f"SELECT 1 FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}'"
    cursor.execute(check_query)
    result = cursor.fetchone()
    
    if result:
        # Table exists, drop the table
        drop_query = f"DROP TABLE {table_name}"
        cursor.execute(drop_query)
        print(f"Table {table_name} dropped.")
    else:
        # Table does not exist, pass
        print(f"Table {table_name} does not exist. Skipping.")

    # Commit the changes to the database
    conn.commit()


# Create a cursor object
cursor = conn.cursor()

# Drop the PositionHistory table if it exists
drop_table(cursor, "PositionHistory")

# Create a cursor object
cursor = conn.cursor()

# Drop the Employee table if it exists
drop_table(cursor, "Employee")

Table PositionHistory dropped.
Table Employee dropped.


Query Create Table Employee and PositionHistory

In [5]:
# Connect to the database
conn = pyodbc.connect(connection_string)

# Create a cursor object
cursor = conn.cursor()

# Create the Employee table
employee_columns = """
    Id int IDENTITY(1,1) NOT NULL,
    EmployeeId varchar(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL,
    FullName varchar(100) COLLATE SQL_Latin1_General_CP1_CI_AS NULL,
    BirthDate date NULL,
    Address varchar(500) COLLATE SQL_Latin1_General_CP1_CI_AS NULL,
    CONSTRAINT PK__Employee__3214EC071C4F15EF PRIMARY KEY (Id),
    CONSTRAINT UQ__Employee__7AD04F10AFCF9E0F UNIQUE (EmployeeId)
"""

# Create the Employee table
create_table(cursor, "Employee", employee_columns)

# Create the PositionHistory table
position_history_columns = """
    Id INT IDENTITY(1, 1) PRIMARY KEY,
    PosId VARCHAR(10),
    PosTitle VARCHAR(100),
    EmployeeId VARCHAR(10),
    StartDate DATE,
    EndDate DATE,
    FOREIGN KEY (EmployeeId) REFERENCES Employee (EmployeeId)
"""

# Create the PositionHistory table
create_table(cursor, "PositionHistory", position_history_columns)

Table Employee created.
Table PositionHistory created.


# 2. Insert Data to Tables Employee and Position History

In [6]:
def insert_data(cursor, table_name, columns, values):
    """
    Inserts data into the specified table.

    Args:
        cursor (pyodbc.Cursor): The cursor object to execute the SQL query.
        table_name (str): The name of the table where the data will be inserted.
        columns (str): A string containing the column names separated by commas.
        values (str): A string containing the values to be inserted, in the format of multiple rows enclosed in parentheses.

    Returns:
        None: The function doesn't return any value.

    Raises:
        pyodbc.Error: If an error occurs while executing the SQL query.

    Example:
        insert_data(cursor, "Employee", "Id, EmployeeId, FullName, BirthDate, Address", "(1, '10105001', 'Ali Anton', '1982-08-19', 'Jakarta Utara'), (2, '10105002', 'Rara Siva', '1982-01-01', 'Mandalika')")

    """
    query = f"INSERT INTO {table_name} ({columns}) VALUES {values}"
    cursor.execute(query)
    conn.commit()

In [7]:
# Insert data into the Employee table
employee_columns = "EmployeeId, FullName, BirthDate, Address"
employee_values = """
  ('10105001', 'Ali Anton', '1982-08-19', 'Jakarta Utara'),
  ('10105002', 'Rara Siva', '1982-01-01', 'Mandalika'),
  ('10105003', 'Rini Aini', '1982-02-20', 'Sumbawa Besar'),
  ('10105004', 'Budi', '1982-02-22', 'Mataram Kota')
"""
insert_data(cursor, "Employee", employee_columns, employee_values)

# Insert data into the PositionHistory table
position_history_columns = "PosId, PosTitle, EmployeeId, StartDate, EndDate"
position_history_values = """
  ('50000', 'IT Manager', '10105001', '2022-01-01', '2022-02-28'),
  ('50001', 'IT Sr. Manager', '10105001', '2022-03-01', '2022-12-31'),
  ('50002', 'Programmer Analyst', '10105002', '2022-01-01', '2022-02-28'),
  ('50003', 'Sr. Programmer Analyst', '10105002', '2022-03-01', '2022-12-31'),
  ('50004', 'IT Admin', '10105003', '2022-01-01', '2022-02-28'),
  ('50005', 'IT Secretary', '10105003', '2022-03-01', '2022-12-31')
"""
insert_data(cursor, "PositionHistory", position_history_columns, position_history_values)

# Commit the changes
conn.commit()

## **3. Query to display all employee (EmployeeId, FullName, BirthDate, Address)**

In [8]:
def retrieve_and_print_data(cursor, table_name):
    """
    Retrieve data from a specified table and print the results.

    Args:
        cursor (pyodbc.Cursor): The cursor object for executing SQL queries.
        table_name (str): The name of the table to retrieve data from.

    Returns:
        None

    """
    query = f"SELECT * FROM {table_name};"
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

Data Employee

In [9]:
retrieve_and_print_data(cursor, "azure_db.dbo.Employee")

(1, '10105001', 'Ali Anton', datetime.date(1982, 8, 19), 'Jakarta Utara')
(2, '10105002', 'Rara Siva', datetime.date(1982, 1, 1), 'Mandalika')
(3, '10105003', 'Rini Aini', datetime.date(1982, 2, 20), 'Sumbawa Besar')
(4, '10105004', 'Budi', datetime.date(1982, 2, 22), 'Mataram Kota')


Data Position History

In [10]:
retrieve_and_print_data(cursor, "azure_db.dbo.PositionHistory")

(1, '50000', 'IT Manager', '10105001', datetime.date(2022, 1, 1), datetime.date(2022, 2, 28))
(2, '50001', 'IT Sr. Manager', '10105001', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))
(3, '50002', 'Programmer Analyst', '10105002', datetime.date(2022, 1, 1), datetime.date(2022, 2, 28))
(4, '50003', 'Sr. Programmer Analyst', '10105002', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))
(5, '50004', 'IT Admin', '10105003', datetime.date(2022, 1, 1), datetime.date(2022, 2, 28))
(6, '50005', 'IT Secretary', '10105003', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))


Display All Employee

In [11]:
query = """
WITH LatestPositionHistory AS (
  SELECT
    EmployeeId,
    MAX(StartDate) AS MaxStartDate
  FROM
    PositionHistory
  GROUP BY
    EmployeeId
)
SELECT
  e.EmployeeId,
  e.FullName,
  e.BirthDate,
  e.Address,
  ph.PosId,
  ph.PosTitle,
  ph.StartDate,
  ph.EndDate
FROM
  Employee e
  INNER JOIN LatestPositionHistory lph ON e.EmployeeId = lph.EmployeeId
  INNER JOIN PositionHistory ph ON lph.EmployeeId = ph.EmployeeId AND lph.MaxStartDate = ph.StartDate;
"""
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)

('10105003', 'Rini Aini', datetime.date(1982, 2, 20), 'Sumbawa Besar', '50005', 'IT Secretary', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))
('10105002', 'Rara Siva', datetime.date(1982, 1, 1), 'Mandalika', '50003', 'Sr. Programmer Analyst', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))
('10105001', 'Ali Anton', datetime.date(1982, 8, 19), 'Jakarta Utara', '50001', 'IT Sr. Manager', datetime.date(2022, 3, 1), datetime.date(2022, 12, 31))


# Generate Data to Create ETL

In [12]:
# Initialize Faker library for generating fake data
fake = Faker('id_ID')
fake.add_provider(address)

In [13]:
# Generate dummy employee data

def generate_employee_data(num_records):
    """
    Generate dummy data for documentation purposes.

    Args:
        num_records (int): Number of records to generate.

    Returns:
        pd.DataFrame: Generated dummy data.
    """
    
    employee_data = []
    for _ in range(num_records):
        employee_id = fake.unique.random_number(digits=8)
        full_name = fake.name()
        birth_date = fake.date_of_birth(minimum_age=18, maximum_age=65)
        address = fake.city_name()
        employee_data.append({
            'EmployeeId': employee_id,
            'FullName': full_name,
            'BirthDate': birth_date,
            'Address': address
        })
    return pd.DataFrame(employee_data)

In [14]:
def generate_position_history_data(num_records, employee_ids):
    """
    Generate dummy data for documentation purposes.

    Args:
        num_records (int): Number of records to generate.
        employee_ids (list): List of employee IDs.

    Returns:
        pd.DataFrame: Generated dummy data.
    """
    position_history_data = []

    for employee_id in employee_ids:
        pos_id = fake.unique.random_number(digits=8)
        pos_title = fake.job()

        start_date = fake.date_between(start_date='-10y', end_date='today')
        end_date = fake.date_between(start_date=start_date, end_date='today')

        position_history_data.append({
            'PosId': pos_id,
            'PosTitle': pos_title,
            'EmployeeId': employee_id,
            'StartDate': start_date,
            'EndDate': end_date
        })

    remaining_records = num_records - len(employee_ids)
    if remaining_records > 0:
        for _ in range(remaining_records):
            employee_id = random.choice(employee_ids)
            pos_id = fake.unique.random_number(digits=8)
            pos_title = fake.job()
            start_date = fake.date_between(start_date='-10y', end_date='today')
            end_date = fake.date_between(start_date=start_date, end_date='today')

            position_history_data.append({
                'PosId': pos_id,
                'PosTitle': pos_title,
                'EmployeeId': employee_id,
                'StartDate': start_date,
                'EndDate': end_date
            })

    return pd.DataFrame(position_history_data)

In [15]:
# Generate dummy training data
def generate_training_data(num_records, employee_ids):
    """
    Generate dummy data for documentation purposes.

    Args:
        num_records (int): Number of records to generate.

    Returns:
        pd.DataFrame: Generated dummy data.
    """
    
    training_data = []
    
    for _ in range(num_records):
        employee_id = np.random.choice(employee_ids)

        # Menggunakan metode datetime untuk mendapatkan tanggal awal
        start_date = datetime.now() - timedelta(days=150)
        training_date = pd.to_datetime(np.random.choice(pd.date_range(start=start_date, end=datetime.now())))

        training_name = fake.job()
        training_duration = np.random.randint(10, 120)
        training_location = fake.city_name()

        # Tambahkan durasi pelatihan (dalam hari) ke tanggal pelatihan
        training_date_end = training_date + pd.DateOffset(days=training_duration)

        training_data.append({
            'EmployeeId': employee_id,
            'TrainingDate_Start': training_date,
            'TrainingDate_End': training_date_end,
            'TrainingName': training_name,
            'TrainingDuration': training_duration,
            'TrainingLocation': training_location
        })

    return pd.DataFrame(training_data)

In [16]:
# Generate dummy data for 100 employees
employee_data = generate_employee_data(25)
employee_data

,EmployeeId,FullName,BirthDate,Address
0,67495493,Vanesa Widiastuti,1965-11-28,Batam
1,51947411,"R.A. Agnes Widiastuti, S.Pd",1982-06-21,Semarang
2,94452394,"R.M. Jagapati Riyanti, S.Pt",1960-11-27,Jambi
3,398342,"Vero Wasita, S.Pt",1989-04-21,Tomohon
4,42358720,Ina Rahayu,1979-06-24,Bima
5,81623537,"dr. Tomi Farida, S.I.Kom",1988-03-31,Mataram
6,23597441,"Chelsea Salahudin, M.Kom.",1961-11-26,Kendari
7,3492627,Intan Saptono,1975-12-10,Payakumbuh
8,81960765,Capa Kuswoyo,1958-10-26,Kotamobagu
9,79467254,Jessica Mangunsong,1989-03-15,Bogor


In [17]:
# Extract the employee IDs for training data generation
employee_ids = employee_data['EmployeeId'].tolist()

In [18]:
# Generate dummy training data for 150 records
training_data = generate_training_data(100, employee_ids)
training_data

,EmployeeId,TrainingDate_Start,TrainingDate_End,TrainingName,TrainingDuration,TrainingLocation
0,42358720,2023-03-05 20:47:36.705756,2023-05-06 20:47:36.705756,Historic buildings inspector/conservation officer,62,Tanjungpinang
1,23597441,2023-02-20 20:47:36.707759,2023-06-02 20:47:36.707759,"Designer, television/film set",102,Kendari
2,67495493,2023-05-05 20:47:36.708758,2023-07-29 20:47:36.708758,Financial controller,85,Tual
3,77224852,2023-05-29 20:47:36.709756,2023-08-14 20:47:36.709756,"Therapist, speech and language",77,Kediri
4,23597441,2023-05-15 20:47:36.710758,2023-06-05 20:47:36.710758,Applications developer,21,Serang
...,...,...,...,...,...,...
95,67495493,2023-05-09 20:47:36.812317,2023-06-10 20:47:36.812317,"Horticulturist, amenity",32,Pontianak
96,75035961,2023-04-28 20:47:36.812317,2023-07-08 20:47:36.812317,Midwife,71,Sawahlunto
97,81623537,2023-06-26 20:47:36.812317,2023-07-19 20:47:36.812317,Copy,23,Kota Administrasi Jakarta Barat
98,23597441,2023-05-21 20:47:36.813316,2023-06-16 20:47:36.813316,Artist,26,Denpasar


In [19]:
position_history_data = generate_position_history_data(60, employee_ids)
position_history_data

,PosId,PosTitle,EmployeeId,StartDate,EndDate
0,66582192,Armed forces operational officer,67495493,2017-01-04,2020-07-16
1,47545914,Television/film/video producer,51947411,2020-12-05,2021-03-23
2,21961740,Financial adviser,94452394,2015-05-30,2017-04-04
3,71695707,"Editor, commissioning",398342,2022-03-16,2023-04-01
4,19515518,Technical author,42358720,2018-11-05,2022-05-04
5,8615702,"Therapist, art",81623537,2022-04-06,2023-06-01
6,25581964,"Editor, commissioning",23597441,2020-12-05,2021-06-08
7,3661535,"Scientist, marine",3492627,2023-06-22,2023-07-04
8,1894878,Market researcher,81960765,2016-11-08,2023-05-04
9,51874933,Equities trader,79467254,2016-05-25,2018-12-20


In [20]:
# Save the data to CSV files
employee_data.to_csv('Documents\dummy_employee_data.csv', index=False)
training_data.to_csv('Documents\dummy_training_data.csv', index=False)

**Insert Data**

In [21]:
def insert_data(conn, table_name, data):
    """
    Insert data into a specified table using a DataFrame.

    Args:
        conn (pyodbc.Connection): The database connection object.
        table_name (str): The name of the table to insert data into.
        data (pandas.DataFrame): The DataFrame containing the data to be inserted.

    Returns:
        None

    """
    cursor = conn.cursor()
    cursor.fast_executemany = True  # Optimize the INSERT statement execution

    columns = ', '.join(data.columns)  # List of columns in the DataFrame
    placeholders = ', '.join(['?' for _ in data.columns])  # Placeholder for each value

    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the INSERT statement for each data row
    cursor.executemany(insert_query, [tuple(row) for row in data.values])

    # Commit the changes
    conn.commit()

In [22]:
# the function to insert data into "dbo.Employee" table
employee_table_name = 'dbo.Employee'
insert_data(conn, employee_table_name, employee_data)

# the function to insert data into "dbo.PositionHistory" table
position_history_table_name = 'dbo.PositionHistory'
insert_data(conn, position_history_table_name, position_history_data)